In [19]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 30 10:11:44 2022

@author: roy.369
"""


import numpy as np
import pickle

import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
mpl.rcParams['axes.linewidth'] = 3
import matplotlib as mpl  
mpl.rc('font',family='Times New Roman')
from matplotlib.pyplot import cm
from numpy.polynomial.polynomial import polyfit
from scipy.optimize import curve_fit
from matplotlib.ticker import ScalarFormatter
from scipy.interpolate import CubicSpline

In [20]:
def arr_loc(val,X):
   ind = 0
   for i in range(len(X)):
      if(round(X[i],2) == round(val,2)):
         ind = i
         break
   return ind


def power_fit(x,a1,m):
    return a1*x**m
  
def linear_fit(x,a,b):
    return a+b*x
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

In [21]:
def r_point_upper_half_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_1 = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              r_lab.append(r_pair_1)  
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_1 = r_pair_1+1

    R_relative_1 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_1,r_pair_1

In [22]:
def r_point_full_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_2 = 0
    for i in range(x_span):
        for j in range(x_span):
              r_lab.append(r_pair_2)
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_2 = r_pair_2+1

    R_relative_2 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_2,r_pair_2

In [23]:
def k_point_grid_upper_half_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              K_lab.append(k_pair)  
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


def k_point_grid_full_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
              K_lab.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


In [24]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = r_point_upper_half_grid(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [25]:
def equal_time_density_density_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,dtau,L):

   data_n_zwl = np.array([0.00005,0.002559,0.010701,0.048674,0.160872,0.312911,0.47211,0.622662,0.779081,0.903597,1.009326,1.111819,1.217161])
   data_dd_zwl = np.array([0,0,0,0,0,-0.000003,-0.000026,-0.000084,0.000027,-0.000184,-0.000301,-0.00401,-0.010903])
   data_hh_zwl = np.array([0,-0.00001,-0.000052,0.001214,-0.001939,-0.008694,-0.017157,-0.01637,-0.008918,-0.001769,0.000079,0.000491,0.000052])
   data_hd_zwl = np.array([0,0,0.000001,0.000073,0.000033,0.000177,0.000387,0.001327,0.00133,0.002594,0.003052,0.000845,0.000282])   
   data_dd_zwl_err = np.array([0,0,0,0,0,0.000004,0.000018,0.000046,0.000153,0.00011,0.000156,0.001726,0.00209])
   data_hh_zwl_err = np.array([0,0.000014,0.000239,0.000496,0.001178,0.001777,0.001991,0.002821,0.001547,0.001526,0.00039,0.000415,0.000427])
   data_hd_zwl_err = np.array([0,0,0.000003,0.000045,0.000077,0.000201,0.000215,0.000334,0.000374,0.000592,0.000256,0.00063,0.000757])   
    
   data_nn_zwl = np.subtract(np.add(data_dd_zwl,data_hh_zwl),2*data_hd_zwl)
   data_nn_zwl_err = np.sqrt(np.add(np.add(np.power(data_dd_zwl_err,2),np.power(data_hh_zwl_err,2)),np.power(data_hd_zwl_err,2)))

   Adj_list = adj_mat_calc(N)
    
   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   
   Den_Den_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Den_Den_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))
    
    
   Nden = np.zeros((len(Mu),len(L)))
   Nden_std = np.zeros((len(Mu),len(L)))
       
   Mu_val = np.zeros(len(Mu))
    
   for j in range(len(Mu)):
       Mu_val[j] = float(Mu[j])

   T_val = np.zeros(len(L))
   for k in range(len(L)):
       T_val[k] = 1/(float(dtau)*float(L[k]))
       
   for j in range(len(Mu)):
       for k in range(len(L)):
                       
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,U,dtau,Mu[j],dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[j][k] = sys_measure_avg['Density averaged']
           Nden_std[j][k] = sys_measure_std['Density standard deviation']
           
           Text_dir_den_den_conn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_connected_correlation_functions'%(Text_dir_main_corr,N,U,dtau,Mu[j],dtau,L[k])

           filename_den_den_conn_corr = '%s/Equal_time_Density_Density_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_conn_corr,N,U,Mu[j],dtau,L[k])
           Den_Den_conn_corr[j,k,:],Den_Den_conn_corr_std[j,k,:] = np.loadtxt(filename_den_den_conn_corr,unpack = 'True', usecols = [1,2])
    
 

   Den_Den_conn_corr_n0 = np.zeros((len(Mu),len(L)))
   Den_Den_conn_corr_n1 = np.zeros((len(Mu),len(L)))
   Den_Den_conn_corr_n2 = np.zeros((len(Mu),len(L)))
   Den_Den_conn_corr_n3 = np.zeros((len(Mu),len(L)))
    
   Den_Den_conn_corr_n0_std = np.zeros((len(Mu),len(L)))
   Den_Den_conn_corr_n1_std = np.zeros((len(Mu),len(L)))
   Den_Den_conn_corr_n2_std = np.zeros((len(Mu),len(L)))
   Den_Den_conn_corr_n3_std = np.zeros((len(Mu),len(L)))

    
   for j in range(len(Mu)):
       for k in range(len(L)):


           Den_Den_conn_corr_n0[j][k] = Den_Den_conn_corr[j][k][0]
           Den_Den_conn_corr_n1[j][k] = Den_Den_conn_corr[j][k][1]
           Den_Den_conn_corr_n2[j][k] = Den_Den_conn_corr[j][k][2]
           Den_Den_conn_corr_n3[j][k] = Den_Den_conn_corr[j][k][3]
           
           Den_Den_conn_corr_n0_std[j][k] = Den_Den_conn_corr_std[j][k][0]
           Den_Den_conn_corr_n1_std[j][k] = Den_Den_conn_corr_std[j][k][1]
           Den_Den_conn_corr_n2_std[j][k] = Den_Den_conn_corr_std[j][k][2]
           Den_Den_conn_corr_n3_std[j][k] = Den_Den_conn_corr_std[j][k][3]

   color_1 = iter(['red','orange','blue'])
   plt.figure(figsize = (25,20))
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 120)
   plt.yticks(fontsize = 120)
   for k in range(len(T_val)):
       c_1 = next(color_1)
       plt.errorbar(Nden[:,k],Den_Den_conn_corr_n1[:,k]*10**3,yerr = Den_Den_conn_corr_n1_std[:,k]*10**3,c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(round(T_val[k],3)))
   plt.errorbar(data_n_zwl,data_nn_zwl*10**3,yerr = data_nn_zwl_err*10**3,c = 'black',marker = "^",markersize = 20, elinewidth = 3, capsize = 5, label = "Expt")   
   plt.grid(True,which='both')
   plt.legend(loc = 'best',ncol = 2, fontsize = 80)
   plt.axhline(y = 0, color = 'black', linestyle = 'dashed', linewidth = 2)  
   plt.axvline(x = 1, color = 'black', linestyle = 'dashed', linewidth = 2) 
   plt.tight_layout()
   plt.savefig("%s/Density_density_correlations_experimental_compare.png"%Graph_dir)
   plt.close()
  

In [26]:
def equal_time_holon_doublon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,dtau,L):

   data_n_zwl = np.array([0.00005,0.002559,0.010701,0.048674,0.160872,0.312911,0.47211,0.622662,0.779081,0.903597,1.009326,1.111819,1.217161])
   data_hd_zwl = np.array([0,0,0.000001,0.000073,0.000033,0.000177,0.000387,0.001327,0.00133,0.002594,0.003052,0.000845,0.000282])
   data_hd_zwl_err = np.array([0,0,0.000003,0.000045,0.000077,0.000201,0.000215,0.000334,0.000374,0.000592,0.000256,0.00063,0.000757])
   
   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

 

   Dbn_Dbn_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Dbn_Dbn_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))
   
   Den_Dbn_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Den_Dbn_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))
   
   Hln_Dbn_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Hln_Dbn_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))
   
   Nden = np.zeros((len(Mu),len(L)))
   Nden_std = np.zeros((len(Mu),len(L)))

       
   T_val = np.zeros(len(L))
   for k in range(len(L)):
       T_val[k] = 1/(float(dtau)*float(L[k]))
       
   for j in range(len(Mu)):
       for k in range(len(L)):
                       
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,U,dtau,Mu[j],dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[j][k] = sys_measure_avg['Density averaged']
           Nden_std[j][k] = sys_measure_std['Density standard deviation']

           Text_dir_dbn_dbn_conn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Doublon_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,U,dtau,Mu[j],dtau,L[k])
           Text_dir_den_dbn_conn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,U,dtau,Mu[j],dtau,L[k])


           filename_dbn_dbn_conn_corr = '%s/Equal_time_Doublon_doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_dbn_dbn_conn_corr,N,U,Mu[j],dtau,L[k])
           Dbn_Dbn_conn_corr[j,k,:],Dbn_Dbn_conn_corr_std[j,k,:] = np.loadtxt(filename_dbn_dbn_conn_corr,unpack = 'True', usecols = [1,2])

           filename_den_dbn_conn_corr = '%s/Equal_time_Density_Doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_dbn_conn_corr,N,U,Mu[j],dtau,L[k])
           Den_Dbn_conn_corr[j,k,:],Den_Dbn_conn_corr_std[j,k,:] = np.loadtxt(filename_den_dbn_conn_corr,unpack = 'True', usecols = [1,2])


           Hln_Dbn_conn_corr[j,k,:] = np.subtract(Dbn_Dbn_conn_corr[j,k,:],Den_Dbn_conn_corr[j,k,:])
           Hln_Dbn_conn_corr_std[j,k,:] = np.sqrt(np.add(np.power(Dbn_Dbn_conn_corr_std[j,k,:],2),np.power(Den_Dbn_conn_corr_std[j,k,:],2)))


    
   Hln_Dbn_conn_corr_n0 = np.zeros((len(Mu),len(L)))
   Hln_Dbn_conn_corr_n1 = np.zeros((len(Mu),len(L)))
   Hln_Dbn_conn_corr_n2 = np.zeros((len(Mu),len(L)))
   Hln_Dbn_conn_corr_n3 = np.zeros((len(Mu),len(L)))
    
   Hln_Dbn_conn_corr_n0_std = np.zeros((len(Mu),len(L)))
   Hln_Dbn_conn_corr_n1_std = np.zeros((len(Mu),len(L)))
   Hln_Dbn_conn_corr_n2_std = np.zeros((len(Mu),len(L)))
   Hln_Dbn_conn_corr_n3_std = np.zeros((len(Mu),len(L)))


   for j in range(len(Mu)):
       for k in range(len(L)):

           Hln_Dbn_conn_corr_n0[j][k] = Hln_Dbn_conn_corr[j][k][0]
           Hln_Dbn_conn_corr_n1[j][k] = Hln_Dbn_conn_corr[j][k][1]
           Hln_Dbn_conn_corr_n2[j][k] = Hln_Dbn_conn_corr[j][k][2]
           Hln_Dbn_conn_corr_n3[j][k] = Hln_Dbn_conn_corr[j][k][3]
           
           Hln_Dbn_conn_corr_n0_std[j][k] = Hln_Dbn_conn_corr_std[j][k][0]
           Hln_Dbn_conn_corr_n1_std[j][k] = Hln_Dbn_conn_corr_std[j][k][1]
           Hln_Dbn_conn_corr_n2_std[j][k] = Hln_Dbn_conn_corr_std[j][k][2]
           Hln_Dbn_conn_corr_n3_std[j][k] = Hln_Dbn_conn_corr_std[j][k][3]

   color_1 = iter(['red','orange','blue'])
   plt.figure(figsize = (25,20))
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 120)
   plt.yticks([0.000,1,2,3],fontsize = 120)
   for k in range(len(T_val)):
       c_1 = next(color_1)
       plt.errorbar(Nden[:,k],Hln_Dbn_conn_corr_n1[:,k]*10**3,yerr = Hln_Dbn_conn_corr_n1_std[:,k]*10**3,c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(round(T_val[k],3)))
   plt.errorbar(data_n_zwl,data_hd_zwl*10**3,yerr = data_hd_zwl_err*10**3,c = 'black',marker = "^",markersize = 20, elinewidth = 3, capsize = 5, label = "Expt")   
   plt.grid(True,which='both')
   plt.legend(loc = 'best',fontsize = 80)
   plt.axhline(y = 0, color = 'black', linestyle = 'dashed', linewidth = 2)  
   plt.axvline(x = 1, color = 'black', linestyle = 'dashed', linewidth = 2) 
   plt.tight_layout()
   plt.savefig("%s/Holon_doublon_correlations_experimental_compare.png"%Graph_dir)
   plt.close()          


In [27]:
def equal_time_moment_moment_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,dtau,L):

   data_n_zwl = np.array([0.00005,0.002559,0.010701,0.048674,0.160872,0.312911,0.47211,0.622662,0.779081,0.903597,1.009326,1.111819,1.217161])
   data_mm_zwl = np.array([0,-0.00001,-0.000049,0.001361,-0.001873,-0.008343,-0.016409,-0.013801,-0.00623,0.003235,0.005881,-0.001829,-0.010288])
   data_mm_zwl_err = np.array([0,0.000014,0.000238,0.000474,0.001127,0.001764,0.001946,0.002866,0.001595,0.001606,0.000776,0.002612,0.002776])

   Adj_list = adj_mat_calc(N)
    
   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   M2_M2_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   M2_M2_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))
  
   Nden = np.zeros((len(Mu),len(L)))
   Nden_std = np.zeros((len(Mu),len(L)))
       
   T_val = np.zeros(len(L))
   for k in range(len(L)):
       T_val[k] = 1/(float(dtau)*float(L[k]))

    
   for j in range(len(Mu)):
       for k in range(len(L)):
               
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,U,dtau,Mu[j],dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[j][k] = sys_measure_avg['Density averaged']
           Nden_std[j][k] = sys_measure_std['Density standard deviation']
              
           Text_dir_m2_m2_conn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Moment_moment_connected_correlation_functions'%(Text_dir_main_corr,N,U,dtau,Mu[j],dtau,L[k])

           filename_m2_m2_conn_corr = '%s/Equal_time_Moment_Moment_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_m2_m2_conn_corr,N,U,Mu[j],dtau,L[k])
           M2_M2_conn_corr[j,k,:],M2_M2_conn_corr_std[j,k,:] = np.loadtxt(filename_m2_m2_conn_corr,unpack = 'True', usecols = [1,2])


   M2_M2_conn_corr_n0 = np.zeros((len(Mu),len(L)))
   M2_M2_conn_corr_n1 = np.zeros((len(Mu),len(L)))
   M2_M2_conn_corr_n2 = np.zeros((len(Mu),len(L)))
   M2_M2_conn_corr_n3 = np.zeros((len(Mu),len(L)))
    
   M2_M2_conn_corr_n0_std = np.zeros((len(Mu),len(L)))
   M2_M2_conn_corr_n1_std = np.zeros((len(Mu),len(L)))
   M2_M2_conn_corr_n2_std = np.zeros((len(Mu),len(L)))
   M2_M2_conn_corr_n3_std = np.zeros((len(Mu),len(L)))


   for j in range(len(Mu)):
       for k in range(len(L)):

           M2_M2_conn_corr_n0[j][k] = M2_M2_conn_corr[j][k][0]
           M2_M2_conn_corr_n1[j][k] = M2_M2_conn_corr[j][k][1]
           M2_M2_conn_corr_n2[j][k] = M2_M2_conn_corr[j][k][2]
           M2_M2_conn_corr_n3[j][k] = M2_M2_conn_corr[j][k][3]

   color_1 = iter(['red','orange','blue'])
   plt.figure(figsize = (25,20))
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 120)
   plt.yticks(fontsize = 120)
   for k in range(len(T_val)):
       c_1 = next(color_1)
       plt.errorbar(Nden[:,k],M2_M2_conn_corr_n1[:,k]*10**3,yerr = M2_M2_conn_corr_n1_std[:,k]*10**3,c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(round(T_val[k],3)))
   plt.errorbar(data_n_zwl,data_mm_zwl*10**3,yerr = data_mm_zwl_err*10**3,c = 'black',marker = "^",markersize = 20, elinewidth = 3, capsize = 5, label = "Expt")   
   plt.grid(True,which='both')
   plt.axhline(y = 0, color = 'black', linestyle = 'dashed', linewidth = 2)  
   plt.axvline(x = 1, color = 'black', linestyle = 'dashed', linewidth = 2)  
   plt.legend(loc = 'best',fontsize = 80)
   plt.tight_layout()
   plt.savefig("%s/Moment_moment_correlations_experimental_compare.png"%Graph_dir)
   plt.close()


In [28]:
def equal_time_holon_holon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,dtau,L):

   data_n_zwl = np.array([0.00005,0.002559,0.010701,0.048674,0.160872,0.312911,0.47211,0.622662,0.779081,0.903597,1.009326,1.111819,1.217161])
   data_hh_zwl = np.array([0,-0.00001,-0.000052,0.001214,-0.001939,-0.008694,-0.017157,-0.01637,-0.008918,-0.001769,0.000079,0.000491,0.000052])
   data_hh_zwl_err = np.array([0,0.000014,0.000239,0.000496,0.001178,0.001777,0.001991,0.002821,0.001547,0.001526,0.00039,0.000415,0.000427])


   Adj_list = adj_mat_calc(N)
    
   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   Den_Den_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Den_Den_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))
   
   Den_Dbn_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Den_Dbn_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))

   Dbn_Dbn_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Dbn_Dbn_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))
    
   Hln_Hln_conn_corr = np.zeros((len(Mu),len(L),len(r_2_unq)))
   Hln_Hln_conn_corr_std = np.zeros((len(Mu),len(L),len(r_2_unq)))   
   
   Nden = np.zeros((len(Mu),len(L)))
   Nden_std = np.zeros((len(Mu),len(L)))

       
   for j in range(len(Mu)):
       for k in range(len(L)):
                
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,U,dtau,Mu[j],dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                    sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,U,Mu[j],dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                    sys_measure_std = pickle.load(infile)

           Nden[j][k] = sys_measure_avg['Density averaged']
           Nden_std[j][k] = sys_measure_std['Density standard deviation']  
               
           Text_dir_den_den_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_connected_correlation_functions'%(Text_dir_main_corr,N,U,dtau,Mu[j],dtau,L[k])
           
           filename_den_den_corr = '%s/Equal_time_Density_Density_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_corr,N,U,Mu[j],dtau,L[k])
           Den_Den_conn_corr[j,k,:],Den_Den_conn_corr_std[j,k,:] = np.loadtxt(filename_den_den_corr,unpack = 'True', usecols = [1,2])

           Text_dir_dbn_dbn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Doublon_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,U,dtau,Mu[j],dtau,L[k])
           
           filename_dbn_dbn_corr = '%s/Equal_time_Doublon_Doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_dbn_dbn_corr,N,U,Mu[j],dtau,L[k])
           Dbn_Dbn_conn_corr[j,k,:],Dbn_Dbn_conn_corr_std[j,k,:] = np.loadtxt(filename_dbn_dbn_corr,unpack = 'True', usecols = [1,2])

           Text_dir_den_dbn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,U,dtau,Mu[j],dtau,L[k])
           
           filename_den_dbn_corr = '%s/Equal_time_Density_Doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_dbn_corr,N,U,Mu[j],dtau,L[k])
           Den_Dbn_conn_corr[j,k,:],Den_Dbn_conn_corr_std[j,k,:] = np.loadtxt(filename_den_dbn_corr,unpack = 'True', usecols = [1,2])

           Hln_Hln_conn_corr[j,k,:] = np.subtract(np.add(Den_Den_conn_corr[j,k,:],Dbn_Dbn_conn_corr[j,k,:]),2*Den_Dbn_conn_corr[j,k,:])
           Hln_Hln_conn_corr_std[j,k,:] = np.sqrt(np.add(np.power(Den_Den_conn_corr_std[j,k,:],2),np.add(np.power(Dbn_Dbn_conn_corr_std[j,k,:],2),4*np.power(Den_Dbn_conn_corr_std[j,k,:],2))))
    

   T_val = np.zeros(len(L))
   for k in range(len(L)):
       T_val[k] = 1/(float(dtau)*float(L[k]))

    
   Hln_Hln_conn_corr_n0 = np.zeros((len(Mu),len(L)))
   Hln_Hln_conn_corr_n1 = np.zeros((len(Mu),len(L)))
   Hln_Hln_conn_corr_n2 = np.zeros((len(Mu),len(L)))
   Hln_Hln_conn_corr_n3 = np.zeros((len(Mu),len(L)))
    
   Hln_Hln_conn_corr_n0_std = np.zeros((len(Mu),len(L)))
   Hln_Hln_conn_corr_n1_std = np.zeros((len(Mu),len(L)))
   Hln_Hln_conn_corr_n2_std = np.zeros((len(Mu),len(L)))
   Hln_Hln_conn_corr_n3_std = np.zeros((len(Mu),len(L)))
 
    
   for j in range(len(Mu)):
       for k in range(len(L)):

           Hln_Hln_conn_corr_n0[j][k] = Hln_Hln_conn_corr[j][k][0]
           Hln_Hln_conn_corr_n1[j][k] = Hln_Hln_conn_corr[j][k][1]
           Hln_Hln_conn_corr_n2[j][k] = Hln_Hln_conn_corr[j][k][2]
           Hln_Hln_conn_corr_n3[j][k] = Hln_Hln_conn_corr[j][k][3]
           
           Hln_Hln_conn_corr_n0_std[j][k] = Hln_Hln_conn_corr_std[j][k][0]
           Hln_Hln_conn_corr_n1_std[j][k] = Hln_Hln_conn_corr_std[j][k][1]
           Hln_Hln_conn_corr_n2_std[j][k] = Hln_Hln_conn_corr_std[j][k][2]
           Hln_Hln_conn_corr_n3_std[j][k] = Hln_Hln_conn_corr_std[j][k][3]

   color_1 = iter(['red','orange','blue'])
   plt.figure(figsize = (25,20))
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 120)
   plt.yticks(fontsize = 120)
   for k in range(len(T_val)):
       c_1 = next(color_1)
       plt.errorbar(Nden[:,k],Hln_Hln_conn_corr_n1[:,k]*10**3,yerr = Hln_Hln_conn_corr_n1_std[:,k]*10**3,c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(round(T_val[k],3)))
   plt.errorbar(data_n_zwl,data_hh_zwl*10**3,yerr = data_hh_zwl_err*10**3,c = 'black',marker = "^",markersize = 20, elinewidth = 3, capsize = 5, label = "Expt")   
   plt.grid(True,which='both')
   plt.legend(loc = 'best',fontsize = 80)
   plt.axhline(y = 0, color = 'black', linestyle = 'dashed', linewidth = 2)  
   plt.axvline(x = 1, color = 'black', linestyle = 'dashed', linewidth = 2) 
   plt.tight_layout()
   plt.savefig("%s/Holon_holon_correlations_experimental_compare.png"%Graph_dir)
   plt.close()

In [29]:
def main():

    N = "10"
    Dtau = "0.05"

    U="11.8" #,"4.50","4.60","4.70"]
    Mu = ["-14.00","-13.50","-13.00","-12.50","-12.00","-11.50","-11.00","-10.50","-10.00","-9.50","-9.00","-8.50","-8.00","-7.50","-7.00",
          "-6.50","-6.00","-5.50","-5.00","-4.50","-4.00","-3.50","-3.00","-2.50","-2.00","-1.50","-1.00","-0.50","0.00","0.50","1.00","1.50",
          "2.00","2.50","3.00","3.50","4.00","4.50","5.00","5.50","6.00","6.50","7.00","7.50","8.00","8.50","9.00","9.50","10.00"]
    Trot = ["12","14","16"]

    Text_dir_main_corr = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_real_space_correlations'%N
    Text_dir_main_eqm = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s'%N
    
    Graph_dir = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s_experimental_compare/Graphs_N_%s_dtau_%s_real_space_correlations_normal_averaged_experimental_compare'%(N,N,Dtau)
    if not os.path.exists(Graph_dir):
        os.makedirs(Graph_dir)
        
    equal_time_density_density_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,Dtau,Trot)
    equal_time_holon_doublon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,Dtau,Trot)
    equal_time_moment_moment_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,Dtau,Trot)
    equal_time_holon_holon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,Dtau,Trot)




In [30]:
main()